In [ ]:
!pip install numpy pandas networkx folium ipyleaflet ipywidgets branca jinja2
# (classic Notebook only)
!jupyter nbextension enable --py widgetsnbextension

In [5]:
import pandas as pd
import numpy as np
import folium
from branca.element import MacroElement
from jinja2 import Template

# ==== PARAMETERS (tune to taste) ====
CSV = "final_speed.csv"

# spatial grid + direction bins
CELL_M = 75
AZIMUTH_BINS = 12
MIN_POINTS = 200
MIN_AVG_SPD_MS = 0.5

# drawing
ARROW_M_PER_MS = 8.0
ARROW_HEAD_ANGLE = 25
ARROW_HEAD_SCALE = 0.35

# ==== 1) load & normalize ====
df = pd.read_csv(CSV).dropna(subset=["lat","lng","spd","azm"]).copy()

# speed units heuristic: if avg > 15, assume km/h -> convert to m/s
spd_mean = df["spd"].mean()
df["spd_ms"] = np.where(spd_mean > 15.0, df["spd"] / 3.6, df["spd"])
df["spd_kmh"] = df["spd_ms"] * 3.6  # keep km/h for display

# ==== helpers ====
def _norm_deg(a):
    a = a % 360.0
    return a if a >= 0 else a + 360.0

def circular_mean_deg(deg_series: pd.Series) -> float:
    rad = np.deg2rad(deg_series.to_numpy() % 360.0)
    s = np.sin(rad).mean()
    c = np.cos(rad).mean()
    ang = np.rad2deg(np.arctan2(s, c))
    return _norm_deg(ang)

def displace_point(lat, lng, az_deg, dist_m, m_per_deg_lat, m_per_deg_lng):
    rad = np.deg2rad(az_deg)
    dlat = (np.cos(rad) * dist_m) / m_per_deg_lat
    dlng = (np.sin(rad) * dist_m) / m_per_deg_lng
    return lat + dlat, lng + dlng

def arrow_segments(lat, lng, az_deg, dist_m, m_per_deg_lat, m_per_deg_lng):
    tail_start = (lat, lng)
    tail_end   = displace_point(lat, lng, az_deg, dist_m, m_per_deg_lat, m_per_deg_lng)
    left_dir  = az_deg + 180 - ARROW_HEAD_ANGLE
    right_dir = az_deg + 180 + ARROW_HEAD_ANGLE
    head_len  = dist_m * ARROW_HEAD_SCALE
    left_pt   = displace_point(*tail_end, left_dir,  head_len, m_per_deg_lat, m_per_deg_lng)
    right_pt  = displace_point(*tail_end, right_dir, head_len, m_per_deg_lat, m_per_deg_lng)
    return tail_start, tail_end, left_pt, right_pt

# ---- speed->color (km/h): green→yellow up to 60, then yellow→red to 100+ ----
def _clamp(x, lo, hi): return max(lo, min(hi, x))
def _lerp(a, b, t):    return a + (b - a) * t
def _rgb_hex(r, g, b): return f"#{int(r):02x}{int(g):02x}{int(b):02x}"

def speed_to_color_kmh(v_kmh: float) -> str:
    v = max(0.0, float(v_kmh))
    # colors (you can tweak):
    green  = (0, 176, 80)      # slow
    yellow = (255, 200, 0)     # medium
    red    = (220, 53, 69)     # fast
    if v <= 60.0:
        t = _clamp(v / 60.0, 0.0, 1.0)
        r = _lerp(green[0], yellow[0], t)
        g = _lerp(green[1], yellow[1], t)
        b = _lerp(green[2], yellow[2], t)
        return _rgb_hex(r, g, b)
    else:
        # blend from 60→100 km/h; ≥100 capped as red
        t = _clamp((v - 60.0) / 40.0, 0.0, 1.0)
        r = _lerp(yellow[0], red[0], t)
        g = _lerp(yellow[1], red[1], t)
        b = _lerp(yellow[2], red[2], t)
        return _rgb_hex(r, g, b)

# ==== 2) grid + azimuth binning ====
lat0 = float(df["lat"].mean())
M_PER_DEG_LAT = 111_320.0
M_PER_DEG_LNG_0 = M_PER_DEG_LAT * np.cos(np.deg2rad(lat0))

df["_lat_q"] = np.floor(df["lat"] * M_PER_DEG_LAT / CELL_M).astype("int64")
df["_lng_q"] = np.floor(df["lng"] * M_PER_DEG_LNG_0 / CELL_M).astype("int64")

BIN_SIZE = 360.0 / AZIMUTH_BINS
df["_dir_q"] = np.floor((df["azm"] % 360.0) / BIN_SIZE).astype("int64")

grp = df.groupby(["_lat_q","_lng_q","_dir_q"], as_index=False).agg(
    cnt=("randomized_id","size"),
    centroid_lat=("lat","mean"),
    centroid_lng=("lng","mean"),
    avg_spd_ms=("spd_ms","mean"),
    avg_spd_kmh=("spd_kmh","mean"),
    avg_azm=("azm", circular_mean_deg),
)

clusters = grp[(grp["cnt"] >= MIN_POINTS) & (grp["avg_spd_ms"] >= MIN_AVG_SPD_MS)].copy()
clusters["m_per_deg_lng"] = M_PER_DEG_LAT * np.cos(np.deg2rad(clusters["centroid_lat"]))

# ==== 3) map of averaged vectors ====
center = [df["lat"].mean(), df["lng"].mean()]
m = folium.Map(location=center, zoom_start=12, control_scale=True)

# small legend (green→yellow→red, 0–60–100+ km/h)
legend = MacroElement()
legend._template = Template("""
{% macro html(this, kwargs) %}
<div style="
 position: fixed; bottom: 20px; left: 20px; z-index: 9999; 
 background: white; padding: 10px 12px; border: 1px solid #888; border-radius: 6px; 
 box-shadow: 0 1px 4px rgba(0,0,0,0.3); font-size: 12px;">
  <div style="font-weight:600; margin-bottom:6px;">Speed (km/h)</div>
  <div style="display:flex; align-items:center; gap:8px;">
    <span>0</span>
    <div style="height:10px; width:140px; background: linear-gradient(90deg, #00b050 0%, #ffc800 60%, #dc3545 100%);"></div>
    <span>100+</span>
  </div>
  <div style="margin-top:4px; color:#555;">Green=slow, Yellow≈60, Red>60</div>
</div>
{% endmacro %}
""")
m.get_root().add_child(legend)

for _, row in clusters.iterrows():
    lat = float(row["centroid_lat"])
    lng = float(row["centroid_lng"])
    avg_spd_ms = float(row["avg_spd_ms"])
    avg_spd_kmh = float(row["avg_spd_kmh"])
    avg_azm = float(row["avg_azm"])

    # arrow length still ∝ average speed in m/s
    dist_m = max(avg_spd_ms, MIN_AVG_SPD_MS) * ARROW_M_PER_MS

    color = speed_to_color_kmh(avg_spd_kmh)

    tail_start, tail_end, left_pt, right_pt = arrow_segments(
        lat, lng, avg_azm, dist_m, M_PER_DEG_LAT, float(row["m_per_deg_lng"])
    )

    folium.PolyLine([tail_start, tail_end], color=color, weight=4, opacity=0.95).add_to(m)
    folium.PolyLine([tail_end, left_pt],  color=color, weight=3, opacity=0.95).add_to(m)
    folium.PolyLine([tail_end, right_pt], color=color, weight=3, opacity=0.95).add_to(m)

    tooltip = (
        f"<b>Avg speed:</b> {avg_spd_kmh:.1f} km/h"
        f"<br><b>Avg azm:</b> {avg_azm:.1f}°"
        f"<br><b>Points:</b> {int(row['cnt'])}"
        f"<br><b>Dir bin:</b> {int(row['_dir_q'])} (≈{int(row['_dir_q']*BIN_SIZE)}–{int((row['_dir_q']+1)*BIN_SIZE)}°)"
        f"<br><b>Cell:</b> {CELL_M} m | bins: {AZIMUTH_BINS}"
    )
    folium.CircleMarker([lat,lng], radius=3, color=color, fill=True, tooltip=tooltip).add_to(m)

m.save("vectors_direction_clusters.html")
print("Done: vectors_direction_clusters.html")

# (optional) export clusters table
clusters_out = clusters.copy()
clusters_out["cell_m"] = CELL_M
clusters_out["az_bins"] = AZIMUTH_BINS
clusters_out["bin_start_deg"] = clusters_out["_dir_q"] * BIN_SIZE
clusters_out["bin_end_deg"] = (clusters_out["_dir_q"] + 1) * BIN_SIZE
clusters_out.to_csv("direction_clusters.csv", index=False)
print("Saved: direction_clusters.csv")

Done: vectors_direction_clusters.html
Saved: direction_clusters.csv


In [ ]:
import math, os, webbrowser
import numpy as np
import pandas as pd
import networkx as nx
import folium

from ipyleaflet import Map, Marker, Polyline, LayerGroup, LayersControl, CircleMarker, WidgetControl
from ipywidgets import HTML, Output, Button
from IPython.display import display, clear_output

# =========================
# Config
# =========================
CLUSTERS_CSV = "direction_clusters.csv"     # change if needed
CLUSTER_DOTS_SAMPLE = 1200                  # how many cluster centers to show
NEIGHBOR_RADIUS_M = 150                     # connect clusters within this radius
MAX_NEIGHBORS_PER_NODE = 16                 # keep graph sparse
MAX_TURN_DEG = 60                           # require forward-ish alignment
TURN_PENALTY = 0.5                          # time *= (1 + TURN_PENALTY*(turn/90)^2)
START_SNAP_RADIUS_M = 600
END_SNAP_RADIUS_M   = 600
OUT_HTML = "fastest_route.html"             # standalone HTML route map

# =========================
# Load clusters
# =========================
clusters = pd.read_csv(CLUSTERS_CSV)

# handle column names (depending on how you saved)
lat_col = "centroid_lat" if "centroid_lat" in clusters.columns else "lat"
lng_col = "centroid_lng" if "centroid_lng" in clusters.columns else "lng"
spd_ms_col = "avg_spd_ms" if "avg_spd_ms" in clusters.columns else ("spd_ms" if "spd_ms" in clusters.columns else None)
azm_col = "avg_azm" if "avg_azm" in clusters.columns else "azm"

if spd_ms_col is None:
    raise ValueError("No speed column found. Expecting 'avg_spd_ms' or 'spd_ms' in CSV.")

clusters["avg_spd_kmh"] = clusters[spd_ms_col] * 3.6

# Normalization range for colors (min=red, max=green)
V_MIN = float(clusters["avg_spd_kmh"].min())
V_MAX = float(clusters["avg_spd_kmh"].max())
if not np.isfinite(V_MIN) or not np.isfinite(V_MAX) or V_MAX <= V_MIN:
    V_MIN, V_MAX = 0.0, 1.0  # fallback to avoid division by zero

# =========================
# Geo helpers
# =========================
M_PER_DEG_LAT = 111_320.0
def meters_per_deg_lng(lat):
    return M_PER_DEG_LAT * np.cos(np.deg2rad(lat))

def dist_m(a_lat, a_lng, b_lat, b_lng):
    m_per_deg_lng = meters_per_deg_lng((a_lat + b_lat) / 2.0)
    dlat = (b_lat - a_lat) * M_PER_DEG_LAT
    dlng = (b_lng - a_lng) * m_per_deg_lng
    return math.hypot(dlat, dlng)

def bearing_deg(a_lat, a_lng, b_lat, b_lng):
    y = math.sin(math.radians(b_lng - a_lng)) * math.cos(math.radians(b_lat))
    x = math.cos(math.radians(a_lat))*math.sin(math.radians(b_lat)) - \
        math.sin(math.radians(a_lat))*math.cos(math.radians(b_lat))*math.cos(math.radians(b_lng - a_lng))
    brng = (math.degrees(math.atan2(y, x)) + 360.0) % 360.0
    return brng

def ang_diff_deg(a, b):
    d = (a - b + 180.0) % 360.0 - 180.0
    return abs(d)

# =========================
# Speed → normalized color (RED -> YELLOW -> GREEN)
# =========================
def _clamp(x, lo, hi): return max(lo, min(hi, x))
def _lerp(a, b, t):    return a + (b - a) * t

def speed_to_color_norm_kmh(v_kmh: float, vmin=V_MIN, vmax=V_MAX) -> str:
    """
    Map speed to a tri-color gradient:
      slowest = RED (#ff0000), middle = YELLOW (#ffc800), fastest = GREEN (#00b050).
    """
    if vmax <= vmin:
        t = 0.5
    else:
        t = _clamp((float(v_kmh) - vmin) / (vmax - vmin), 0.0, 1.0)

    red    = (255,   0,   0)   # #ff0000
    yellow = (255, 200,   0)   # #ffc800
    green  = (  0, 176,  80)   # #00b050

    if t <= 0.5:
        # 0..0.5 : RED -> YELLOW
        tt = t / 0.5
        r = _lerp(red[0],    yellow[0], tt)
        g = _lerp(red[1],    yellow[1], tt)
        b = _lerp(red[2],    yellow[2], tt)
    else:
        # 0.5..1 : YELLOW -> GREEN
        tt = (t - 0.5) / 0.5
        r = _lerp(yellow[0], green[0], tt)
        g = _lerp(yellow[1], green[1], tt)
        b = _lerp(yellow[2], green[2], tt)

    return f"#{int(r):02x}{int(g):02x}{int(b):02x}"

# =========================
# Build directed graph
# =========================
G = nx.DiGraph()
for i, r in clusters.iterrows():
    G.add_node(
        i,
        lat=float(r[lat_col]),
        lng=float(r[lng_col]),
        azm=float(r[azm_col]) % 360.0,
        spd_kmh=float(r["avg_spd_kmh"]),
        cnt=int(r.get("cnt", 1))
    )

node_ids = list(G.nodes)
lats = np.array([G.nodes[n]["lat"] for n in node_ids])
lngs = np.array([G.nodes[n]["lng"] for n in node_ids])

for idx, n in enumerate(node_ids):
    a_lat, a_lng, a_azm, a_spd = G.nodes[n]["lat"], G.nodes[n]["lng"], G.nodes[n]["azm"], G.nodes[n]["spd_kmh"]

    # bounding box prefilter
    m_per_deg_lng = meters_per_deg_lng(a_lat)
    dlat_deg = NEIGHBOR_RADIUS_M / M_PER_DEG_LAT
    dlng_deg = NEIGHBOR_RADIUS_M / m_per_deg_lng
    mask = (abs(lats - a_lat) <= dlat_deg) & (abs(lngs - a_lng) <= dlng_deg)

    cands = [node_ids[j] for j in np.where(mask)[0] if node_ids[j] != n]
    dists = [(m, dist_m(a_lat, a_lng, G.nodes[m]["lat"], G.nodes[m]["lng"])) for m in cands]
    dists = [(m, d) for (m, d) in dists if d <= NEIGHBOR_RADIUS_M]
    dists.sort(key=lambda x: x[1])
    dists = dists[:MAX_NEIGHBORS_PER_NODE]

    for m, d in dists:
        b_lat, b_lng = G.nodes[m]["lat"], G.nodes[m]["lng"]
        fwd = bearing_deg(a_lat, a_lng, b_lat, b_lng)
        turn = ang_diff_deg(fwd, a_azm)
        if turn <= MAX_TURN_DEG:
            v_ms = max(((a_spd + G.nodes[m]["spd_kmh"]) / 2.0) / 3.6, 0.1)
            time_s = d / v_ms * (1.0 + TURN_PENALTY * (turn / 90.0) ** 2)
            G.add_edge(n, m,
                       dist_m=d,
                       turn_deg=turn,
                       speed_kmh=(a_spd + G.nodes[m]["spd_kmh"]) / 2.0,
                       time_s=time_s)

# =========================
# Snap + route
# =========================
def nearest_nodes_for_point(lat, lng, max_radius_m, forward_az=None, max_turn=120):
    m_per_deg_lng = meters_per_deg_lng(lat)
    dlat_deg = max_radius_m / M_PER_DEG_LAT
    dlng_deg = max_radius_m / m_per_deg_lng
    mask = (abs(lats - lat) <= dlat_deg) & (abs(lngs - lng) <= dlng_deg)

    cand = []
    for idx in np.where(mask)[0]:
        nid = node_ids[idx]
        d = dist_m(lat, lng, G.nodes[nid]["lat"], G.nodes[nid]["lng"])
        if d <= max_radius_m:
            if forward_az is not None:
                turn = ang_diff_deg(forward_az, G.nodes[nid]["azm"])
                if turn > max_turn:
                    continue
            cand.append((nid, d))
    cand.sort(key=lambda x: x[1])
    return [n for n, _ in cand[:8]]

def best_path(start_lat, start_lng, end_lat, end_lng):
    desired_start_bearing = bearing_deg(start_lat, start_lng, end_lat, end_lng)
    desired_end_bearing   = bearing_deg(end_lat, end_lng, start_lat, start_lng)

    start_nodes = nearest_nodes_for_point(start_lat, start_lng, START_SNAP_RADIUS_M, desired_start_bearing)
    end_nodes   = nearest_nodes_for_point(end_lat, end_lng, END_SNAP_RADIUS_M,   desired_end_bearing)

    H = G.copy()
    H.add_node("START", lat=start_lat, lng=start_lng, azm=desired_start_bearing, spd_kmh=999.0)
    H.add_node("END",   lat=end_lat,   lng=end_lng,   azm=desired_end_bearing,   spd_kmh=999.0)

    for n in start_nodes:
        d = dist_m(start_lat, start_lng, H.nodes[n]["lat"], H.nodes[n]["lng"])
        v_ms = max(H.nodes[n]["spd_kmh"]/3.6, 0.1)
        turn = ang_diff_deg(bearing_deg(start_lat, start_lng, H.nodes[n]["lat"], H.nodes[n]["lng"]), H.nodes[n]["azm"])
        time_s = d / v_ms * (1 + TURN_PENALTY * (turn/90.0)**2)
        H.add_edge("START", n, time_s=time_s, dist_m=d, speed_kmh=H.nodes[n]["spd_kmh"], turn_deg=turn)

    for n in end_nodes:
        d = dist_m(H.nodes[n]["lat"], H.nodes[n]["lng"], end_lat, end_lng)
        v_ms = max(H.nodes[n]["spd_kmh"]/3.6, 0.1)
        turn = ang_diff_deg(bearing_deg(H.nodes[n]["lat"], H.nodes[n]["lng"], end_lat, end_lng), H.nodes[n]["azm"])
        time_s = d / v_ms * (1 + TURN_PENALTY * (turn/90.0)**2)
        H.add_edge(n, "END", time_s=time_s, dist_m=d, speed_kmh=H.nodes[n]["spd_kmh"], turn_deg=turn)

    path = nx.shortest_path(H, source="START", target="END", weight="time_s", method="dijkstra")
    return H, path

# =========================
# Map UI
# =========================
center = (float(clusters[lat_col].mean()), float(clusters[lng_col].mean()))
m = Map(center=center, zoom=12)
m.add_control(LayersControl())

# legend (normalized, Red -> Yellow -> Green)
legend_html = HTML(
    value=f"""
<div style="background:white; padding:8px 10px; border:1px solid #999; border-radius:6px; font:12px sans-serif;">
  <div style="font-weight:600; margin-bottom:6px;">Speed (km/h) — normalized</div>
  <div style="display:flex; align-items:center; gap:8px;">
    <span>{V_MIN:.1f}</span>
    <div style="height:10px; width:160px; background: linear-gradient(90deg, #ff0000 0%, #ffc800 50%, #00b050 100%);"></div>
    <span>{V_MAX:.1f}</span>
  </div>
  <div style="margin-top:4px; color:#555;">Red = slowest, Yellow = medium, Green = fastest</div>
</div>
"""
)
m.add_control(WidgetControl(widget=legend_html, position="bottomleft"))

ui_out = Output()
display(ui_out)

# Layers
route_layer = LayerGroup()  # holds start/end markers + route polylines
m.add_layer(route_layer)

# scatter cluster centers
def draw_clusters_points(sample=CLUSTER_DOTS_SAMPLE):
    idx = clusters.index if (sample is None or sample >= len(clusters)) else clusters.sample(sample, random_state=42).index
    for i in idx:
        r = clusters.loc[i]
        m.add_layer(CircleMarker(
            location=(float(r[lat_col]), float(r[lng_col])),
            radius=2,
            color="#666",
            fill_color="#666",
            fill_opacity=0.5,
            opacity=0.5
        ))
draw_clusters_points()

# keep last route to export
last_route = {"H": None, "path": None, "start": None, "end": None}

def save_route_to_html():
    if last_route["H"] is None or last_route["path"] is None:
        with ui_out:
            print("No route yet. Click Start and End on the map first.")
        return
    H, path = last_route["H"], last_route["path"]
    s, e = last_route["start"], last_route["end"]

    # build standalone Folium map
    m2 = folium.Map(location=[(s[0]+e[0])/2, (s[1]+e[1])/2], zoom_start=12, control_scale=True)
    folium.Marker(s, icon=folium.Icon(color="green"), tooltip="Start").add_to(m2)
    folium.Marker(e, icon=folium.Icon(color="red"), tooltip="End").add_to(m2)

    for i in range(len(path) - 1):
        a, b = path[i], path[i+1]
        a_lat, a_lng = H.nodes[a]["lat"], H.nodes[a]["lng"]
        b_lat, b_lng = H.nodes[b]["lat"], H.nodes[b]["lng"]
        spd = H.edges[a, b]["speed_kmh"]
        color = speed_to_color_norm_kmh(spd, V_MIN, V_MAX)
        folium.PolyLine([(a_lat,a_lng),(b_lat,b_lng)], color=color, weight=6, opacity=0.95).add_to(m2)

    # add a small legend to the HTML
    legend = folium.map.CustomPane("legend")
    m2.add_child(legend)
    legend_html = f"""
    <div style="position: fixed; bottom: 20px; left: 20px; z-index: 9999;
                background: white; padding: 8px 10px; border: 1px solid #999; border-radius: 6px; font: 12px sans-serif;">
      <div style="font-weight:600; margin-bottom:6px;">Speed (km/h) — normalized</div>
      <div style="display:flex; align-items:center; gap:8px;">
        <span>{V_MIN:.1f}</span>
        <div style="height:10px; width:160px; background: linear-gradient(90deg, #ff0000 0%, #ffc800 50%, #00b050 100%);"></div>
        <span>{V_MAX:.1f}</span>
      </div>
      <div style="margin-top:4px; color:#555;">Red = slowest, Yellow = medium, Green = fastest</div>
    </div>"""
    m2.get_root().html.add_child(folium.Element(legend_html))

    m2.save(OUT_HTML)

    url = "file://" + os.path.abspath(OUT_HTML)
    try:
        webbrowser.open(url)
    except Exception:
        pass

    with ui_out:
        print(f"Saved: {OUT_HTML}")
        print(f"Open (Ctrl+Click): {url}")

# "Open in separate window" button
btn_open = Button(description="Open route in new window", button_style="success")
def on_btn_click(_):
    save_route_to_html()
btn_open.on_click(on_btn_click)
m.add_control(WidgetControl(widget=btn_open, position="topleft"))

# routing & clicks
click_state = {"start": None, "end": None}

def recompute_and_draw():
    route_layer.layers = ()  # clear

    s = click_state["start"]; e = click_state["end"]
    if not s or not e:
        return

    try:
        H, path = best_path(s[0], s[1], e[0], e[1])
    except Exception as ex:
        with ui_out:
            clear_output()
            print("Routing failed:", ex)
        return

    # add start/end markers
    route_layer.add_layer(Marker(location=s))
    route_layer.add_layer(Marker(location=e))

    # draw route, color each edge by its normalized speed
    total_time = 0.0
    total_dist = 0.0

    for i in range(len(path) - 1):
        a, b = path[i], path[i+1]
        a_lat, a_lng = H.nodes[a]["lat"], H.nodes[a]["lng"]
        b_lat, b_lng = H.nodes[b]["lat"], H.nodes[b]["lng"]
        edge = H.edges[a, b]
        total_time += edge["time_s"]
        total_dist += edge["dist_m"]

        color = speed_to_color_norm_kmh(edge["speed_kmh"], V_MIN, V_MAX)
        seg = Polyline(locations=[(a_lat,a_lng),(b_lat,b_lng)], weight=6, opacity=0.95, color=color)
        route_layer.add_layer(seg)

    # remember for export
    last_route["H"] = H
    last_route["path"] = path
    last_route["start"] = s
    last_route["end"] = e

    with ui_out:
        clear_output()
        print(f"Segments: {len(path)-1}, distance: {total_dist/1000:.2f} km, time: {total_time/60:.1f} min")
        print("Click the green button to open the route in a new window.")

def handle_click(**kwargs):
    if kwargs.get("type") == "click":
        lat, lng = kwargs.get("coordinates")
        if click_state["start"] is None:
            click_state["start"] = (lat, lng)
            with ui_out:
                clear_output()
                print(f"Start set: {lat:.6f}, {lng:.6f}. Now click End…")
        elif click_state["end"] is None:
            click_state["end"] = (lat, lng)
            with ui_out:
                print(f"End set: {lat:.6f}, {lng:.6f}. Computing route…")
            recompute_and_draw()
        else:
            # third click resets
            click_state["start"] = None
            click_state["end"] = None
            route_layer.layers = ()
            last_route.update({"H": None, "path": None, "start": None, "end": None})
            with ui_out:
                clear_output()
                print("Reset. Click Start, then End.")

m.on_interaction(handle_click)
m

Output()

Map(center=[51.09108687634974, 71.41774165667171], controls=(ZoomControl(options=['position', 'zoom_in_text', …